In [19]:
!pip install tabulate

In [20]:
import pyodbc
from tabulate import tabulate
print(pyodbc.drivers())


['SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server', 'ODBC Driver 18 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [4]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd

load_dotenv()

def get_connection():
    server = os.getenv("SQL_SERVER")
    database = os.getenv("SQL_DATABASE")
    driver = next((d for d in reversed(pyodbc.drivers()) if "ODBC Driver" in d), None)
    
    conn_str = (
        f"DRIVER={{{driver}}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        "Trusted_Connection=yes;"
        "TrustServerCertificate=yes;"  # Bypass certificate validation
        # "Encrypt=optional;"  # Alternative for some environments
    )
    
    try:
        return pyodbc.connect(conn_str, timeout=10)
    except pyodbc.Error as e:
        print(f"Connection failed: {str(e)}")
        return None

# Senate Committee Activity Analysis Query

This query generates a comprehensive report on committee operations by joining multiple tables in the legislative database. It reveals workload, membership, and voting patterns across all active Senate committees.

## Key Components

### 1. Committee Composition Analysis
- **Membership Totals**: Counts all senators assigned to each committee (`Members`)
- **Partisan Breakdown**: Separates counts by party affiliation (`DemocraticMembers`, `RepublicanMembers`)
- **Committee Metadata**: Includes name and type (Standing/Subcommittee/etc.)

### 2. Legislative Workload Metrics
- **Active Bills**: Counts bills currently being processed by each committee (where `IsCurrentStatus = 1`)
- **Voting Activity**: Tracks total votes taken (`CommitteeVotes`)
- **Timeframe**: Shows date range of committee operations (`FirstSessionDate`, `LastSessionDate`)

### 3. Technical Implementation
- **Multi-Table Join**: Connects committees → bills → senators → votes → dates
- **Conditional Logic**: Uses `CASE WHEN` for party-specific counts
- **Aggregation**: `GROUP BY` consolidates all metrics at committee level
- **Filtering**: Only includes currently active committees (`WHERE dc.IsActive = 1`)
- **Sorting**: Orders results by busiest committees (`ActiveBills DESC`)

## Political Significance
- Reveals which committees handle the most legislation
- Shows partisan balance (which party controls each committee)
- Identifies committees with heavy voting workloads
- Tracks operational timeframes (when committees are most active)

## Note on Data Relationships
⚠️ The join between `FactBillStatus` and `DimCommittee` assumes committee keys match bill keys directly. In most Senate databases, this would require a bridge table like `CommitteeReferrals`.

In [27]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            # Committee Workload Query
            dfComiteeWorkLoad = pd.read_sql("""
SELECT 
    dc.CommitteeName,
    dc.CommitteeType,
    COUNT(DISTINCT CASE WHEN fbs.IsCurrentStatus = 1 THEN fbs.BillKey END) AS ActiveBills,
    COUNT(DISTINCT fca.SenatorKey) AS Members,
    SUM(CASE WHEN ds.PartyAffiliation = 'Democratic' THEN 1 ELSE 0 END) AS DemocraticMembers,
    SUM(CASE WHEN ds.PartyAffiliation = 'Republican' THEN 1 ELSE 0 END) AS RepublicanMembers,
    COUNT(DISTINCT fvr.VoteRecordKey) AS CommitteeVotes,
    MIN(dd.FullDate) AS FirstSessionDate,
    MAX(dd.FullDate) AS LastSessionDate
FROM DimCommittee dc
LEFT JOIN FactBillStatus fbs ON dc.CommitteeKey = fbs.BillKey -- Assuming relationship
LEFT JOIN FactCommitteeAssignment fca ON dc.CommitteeKey = fca.CommitteeKey AND fca.AssignmentEndDate IS NULL
LEFT JOIN DimSenator ds ON fca.SenatorKey = ds.SenatorKey
LEFT JOIN FactVoteRecord fvr ON dc.CommitteeKey = fvr.CommitteeKey
LEFT JOIN DimDate dd ON fvr.DateKey = dd.DateKey
WHERE dc.IsActive = 1
GROUP BY dc.CommitteeName, dc.CommitteeType
ORDER BY ActiveBills DESC;
            """, conn)
            print("Senate committee activity:")
            print(tabulate(dfComiteeWorkLoad, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Senate committee activity:
+----------------------------------------+-----------------+---------------+-----------+---------------------+---------------------+------------------+--------------------+-------------------+
| CommitteeName                          | CommitteeType   |   ActiveBills |   Members |   DemocraticMembers |   RepublicanMembers |   CommitteeVotes | FirstSessionDate   | LastSessionDate   |
|----------------------------------------+-----------------+---------------+-----------+---------------------+---------------------+------------------+--------------------+-------------------|
| Appropriations                         | Standing        |             1 |         3 |                 252 |                 168 |                7 | 2024-01-20         | 2024-03-10        |
| Finance                                | Standing        |             1 |         3 |                  44 |                  22 |                2 | 2024-01-20         | 2024-01-20        |
| Foreig

C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\614370318.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfComiteeWorkLoad = pd.read_sql("""


# Congressional Work Calendar Analysis Query

This query analyzes legislative productivity patterns by month, tracking how the Senate's time is divided between session days, recess periods, and holidays while measuring legislative output.

## Key Components

### 1. Timeframe Analysis
- **Session Days**: Counts actual working days (`IsCongressionalSessionDay = 1`)
- **Recess Periods**: Identifies non-session weekdays (`IsCongressionalSessionDay = 0 AND IsWeekday = 1`)
- **Holidays**: Flags official non-working days

### 2. Legislative Output Metrics
- **Bills Introduced**: Counts new legislation (`StatusCode = 'INTRO'`)
- **Votes Taken**: Tracks all floor/committee votes
- **Bills Passed**: Measures successful legislation (`StatusCode = 'PASS'`)

### 3. Technical Implementation
- **Date-Driven**: Anchored to `DimDate` table for calendar intelligence
- **Smart Joins**: 
  - Links bill introductions to specific dates
  - Connects votes to their occurrence dates
- **Conditional Counting**: Uses `CASE WHEN` for nuanced categorization
- **Time Filtering**: Focuses on 118th Congress (2023-2025)
- **Sorting**: Chronological order by year/month

## Political Significance
- Reveals legislative "pulse" - when most work occurs
- Shows correlation between session days and productivity
- Identifies seasonal patterns (e.g., summer recess slowdown)
- Helps predict IT resource needs (high-activity periods)

## Implementation Notes
- The `LEFT JOIN` ensures we count dates even with no legislative activity
- `DISTINCT` prevents double-counting bills with multiple status changes
- MonthNumber in `GROUP BY` ensures proper chronological ordering

In [23]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            
            dfCongressionalWorkPatterns = pd.read_sql("""
SELECT 
    dd.MonthName,
    dd.Year,
    dd.CongressionalSession,
    COUNT(DISTINCT CASE WHEN dd.IsCongressionalSessionDay = 1 THEN dd.FullDate END) AS SessionDays,
    COUNT(DISTINCT fbs.BillKey) AS BillsIntroduced,
    COUNT(DISTINCT fvr.VoteRecordKey) AS VotesTaken,
    COUNT(DISTINCT CASE WHEN fbs.StatusCode = 'PASS' THEN fbs.BillKey END) AS BillsPassed,
    COUNT(DISTINCT CASE WHEN dd.IsHoliday = 1 THEN dd.FullDate END) AS Holidays,
    COUNT(DISTINCT CASE WHEN dd.IsCongressionalSessionDay = 0 AND dd.IsWeekday = 1 THEN dd.FullDate END) AS RecessWeekdays
FROM DimDate dd
LEFT JOIN FactBillStatus fbs ON dd.DateKey = fbs.DateKey AND fbs.StatusCode = 'INTRO'
LEFT JOIN FactVoteRecord fvr ON dd.DateKey = fvr.DateKey
WHERE dd.FullDate BETWEEN '2023-01-01' AND '2025-12-31'
GROUP BY dd.MonthName, dd.Year, dd.CongressionalSession, dd.MonthNumber
ORDER BY dd.Year, dd.MonthNumber;
            """, conn)
            print("Congressional Work Patterns by Month:")
            print(tabulate(dfCongressionalWorkPatterns, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Congressional Work Patterns by Month:
+-------------+--------+------------------------+---------------+-------------------+--------------+---------------+------------+------------------+
| MonthName   |   Year |   CongressionalSession |   SessionDays |   BillsIntroduced |   VotesTaken |   BillsPassed |   Holidays |   RecessWeekdays |
|-------------+--------+------------------------+---------------+-------------------+--------------+---------------+------------+------------------|
| January     |   2024 |                    118 |            16 |                 3 |           35 |             0 |          2 |                7 |
| February    |   2024 |                    118 |            16 |                 2 |            7 |             0 |          1 |                5 |
| March       |   2024 |                    118 |            16 |                 2 |            7 |             0 |          0 |                5 |
| April       |   2024 |                    118 |            17 |   

C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\2760538339.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfCongressionalWorkPatterns = pd.read_sql("""


# Bipartisan Bill Support Analysis Query

This query identifies bills with cross-party support and measures their legislative success, providing insights into partisan cooperation patterns.

## Key Metrics

### 1. Cosponsorship Analysis
- **Party Breakdown**: Counts cosponsors by party (Democratic/Republican/Independent)
- **Bipartisan Flag**: `IsBipartisan` indicates multi-party support
- **Opposition Support**: Calculates percentage of cosponsors from opposing party

### 2. Legislative Outcomes
- **Vote Results**: Shows final disposition (Passed/Failed)
- **Active Bills**: Only considers currently active legislation (`IsActive = 1`)

### 3. Technical Implementation
- **Multi-Level Joins**: Links bills → sponsors → senators → vote results
- **Nested Subquery**: Isolates substantive votes (not procedural)
- **Conditional Aggregation**: `CASE WHEN` for party-specific counts
- **Percentage Math**: Safely calculates opposition support with `NULLIF`
- **Filtering**: Only bills with bipartisan cosponsors (`HAVING COUNT... > 1`)
- **Sorting**: Shows most bipartisan bills first

## Political Significance
- Reveals which issues generate cross-party cooperation
- Measures opposition party engagement levels
- Identifies successful bipartisan strategies
- Highlights subjects with unusual alliance patterns

## Implementation Notes
- The subquery filters for only substantive votes (final passages, not amendments)
- `NULLIF` prevents divide-by-zero errors for bills with no cosponsors
- `DISTINCT` ensures senators aren't double-counted if listed multiple times

In [22]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            # Committee Workload Query
            dfCrossPartyCollaboration = pd.read_sql("""
SELECT 
    db.BillNumber,
    db.Title,
    db.PrimarySubject,
    COUNT(DISTINCT CASE WHEN ds.PartyAffiliation = 'Democratic' THEN ds.SenatorKey END) AS DemocraticCosponsors,
    COUNT(DISTINCT CASE WHEN ds.PartyAffiliation = 'Republican' THEN ds.SenatorKey END) AS RepublicanCosponsors,
    COUNT(DISTINCT CASE WHEN ds.PartyAffiliation NOT IN ('Democratic', 'Republican') THEN ds.SenatorKey END) AS IndependentCosponsors,
    CAST(COUNT(DISTINCT CASE WHEN ds.PartyAffiliation = 'Republican' THEN ds.SenatorKey END) * 100.0 / 
        NULLIF(COUNT(DISTINCT ds.SenatorKey), 0) AS DECIMAL(5,2)) AS PctOppositionPartySupport,
    fvr.VoteResult,
    CASE WHEN COUNT(DISTINCT ds.PartyAffiliation) > 1 THEN 1 ELSE 0 END AS IsBipartisan
FROM DimBill db
JOIN FactBillSponsorship fbs ON db.BillKey = fbs.BillKey
JOIN DimSenator ds ON fbs.SenatorKey = ds.SenatorKey
LEFT JOIN (
    SELECT BillKey, VoteResult 
    FROM FactVoteRecord 
    WHERE VoteTypeKey IN (SELECT VoteTypeKey FROM DimVoteType WHERE VoteCategory = 'Substantive')
    GROUP BY BillKey, VoteResult
) fvr ON db.BillKey = fvr.BillKey
WHERE db.IsActive = 1
GROUP BY db.BillNumber, db.Title, db.PrimarySubject, fvr.VoteResult
HAVING COUNT(DISTINCT ds.PartyAffiliation) > 1
ORDER BY PctOppositionPartySupport DESC;
            """, conn)
            print("Cross-Party Collaboration Metrics:")
            print(tabulate(dfCrossPartyCollaboration, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Cross-Party Collaboration Metrics:
+--------------+------------------------------+------------------+------------------------+------------------------+-------------------------+-----------------------------+--------------+----------------+
| BillNumber   | Title                        | PrimarySubject   |   DemocraticCosponsors |   RepublicanCosponsors |   IndependentCosponsors |   PctOppositionPartySupport | VoteResult   |   IsBipartisan |
|--------------+------------------------------+------------------+------------------------+------------------------+-------------------------+-----------------------------+--------------+----------------|
| S.101        | Small Business Relief Act    | Commerce         |                      1 |                      3 |                       0 |                       75    | Passed       |              1 |
| S.456        | Healthcare Affordability Act | Health           |                      1 |                      3 |                       0 |   

C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\996964640.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfCrossPartyCollaboration = pd.read_sql("""


# Committee Workload Analysis Query

**Purpose**: Shows each committee's legislative volume and success rate by counting bills referred, votes taken, and bills passed.

## Key Metrics
- `BillsReferred`: Total bills assigned to committee  
- `VotesTaken`: Formal votes conducted  
- `BillsPassed`: Bills successfully advanced  

## Technical Notes
- **Joins**: Committees → Votes → Bills → Bill Status  
- `LEFT JOIN`: Includes all committees (even inactive ones)  
- `COUNT(DISTINCT)`: Prevents duplicate counting  
- `ORDER BY`: Ranks busiest committees first  

**Use Case**: Helps identify overburdened committees and measure legislative efficiency.

In [ ]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            # Committee Workload Query
            dfActiveCommittees = pd.read_sql("""
SELECT 
    c.CommitteeName,
    COUNT(DISTINCT b.BillKey) AS BillsReferred,
    COUNT(DISTINCT v.VoteRecordKey) AS VotesTaken,
    COUNT(DISTINCT CASE WHEN bs.StatusCode = 'PASS' THEN b.BillKey END) AS BillsPassed
FROM DimCommittee c
LEFT JOIN FactVoteRecord v ON c.CommitteeKey = v.CommitteeKey
LEFT JOIN DimBill b ON v.BillKey = b.BillKey
LEFT JOIN FactBillStatus bs ON b.BillKey = bs.BillKey
GROUP BY c.CommitteeName
ORDER BY BillsReferred DESC;
            """, conn)
            print("Most Active Committees:")
            print(tabulate(dfActiveCommittees, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Most Active Committees:
+----------------------------------------+-----------------+--------------+---------------+
| CommitteeName                          |   BillsReferred |   VotesTaken |   BillsPassed |
|----------------------------------------+-----------------+--------------+---------------|
| Appropriations                         |               0 |            7 |             0 |
| Constitution                           |               0 |            0 |             0 |
| Defense Appropriations                 |               0 |            0 |             0 |
| Europe and Regional Security           |               0 |            0 |             0 |
| Finance                                |               0 |            2 |             0 |
| Foreign Relations                      |               0 |            2 |             0 |
| Health, Education, Labor, and Pensions |               0 |            2 |             0 |
| Judiciary                              |              

C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\2198318766.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfActiveCommittees = pd.read_sql("""


# Party Legislative Effectiveness Query

**Purpose**: Compares how successful each party is at passing legislation and maintaining party unity.

## Key Metrics
- `BillsSponsored`: Number of bills introduced by party members  
- `BillsPassed`: Bills that became law  
- `PassageRate`: Percentage of sponsored bills that passed  
- `AvgVotesPerSenator`: Average voting participation  
- `PartyUnityScore`: How often party members vote together  

## Technical Notes
- **Joins**: Senators → Bill Sponsorships → Dates → Bill Status → Votes  
- `LEFT JOIN`: Includes all sponsors (even if bills didn't pass)  
- `NULLIF`: Prevents divide-by-zero in PassageRate  
- `ORDER BY`: Groups by Congress session then most active parties  

**Use Case**: Measures party effectiveness and discipline in passing legislation.

In [ ]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            # Committee Workload Query
            dfSenateProductivity = pd.read_sql("""
SELECT 
    dd.CongressionalSession,
    ds.PartyAffiliation,
    COUNT(DISTINCT fb.BillKey) AS BillsSponsored,
    COUNT(DISTINCT fbs.BillKey) AS BillsPassed,
    CAST(COUNT(DISTINCT fbs.BillKey) * 100.0 / NULLIF(COUNT(DISTINCT fb.BillKey), 0) 
        AS DECIMAL(5,2)) AS PassageRate,
    AVG(fv.VotesCast) AS AvgVotesPerSenator,
    CAST(SUM(CASE WHEN fvr.VoteCast = 'Yea' THEN 1 ELSE 0 END) * 100.0 / 
        COUNT(fvr.VoteRecordKey) AS DECIMAL(5,2)) AS PartyUnityScore
FROM DimSenator ds
JOIN FactBillSponsorship fb ON ds.SenatorKey = fb.SenatorKey
JOIN DimDate dd ON fb.DateKey = dd.DateKey
LEFT JOIN FactBillStatus fbs ON fb.BillKey = fbs.BillKey AND fbs.StatusCode = 'PASS'
LEFT JOIN (
    SELECT SenatorKey, COUNT(*) AS VotesCast 
    FROM FactVoteRecord 
    GROUP BY SenatorKey
) fv ON ds.SenatorKey = fv.SenatorKey
LEFT JOIN FactVoteRecord fvr ON ds.SenatorKey = fvr.SenatorKey
WHERE fb.IsPrimarySponsor = 1
GROUP BY dd.CongressionalSession, ds.PartyAffiliation
ORDER BY dd.CongressionalSession, BillsSponsored DESC;
            """, conn)
            print("Senate Productivity:")
            print(tabulate(dfSenateProductivity, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Senate Productivity:
+------------------------+--------------------+------------------+---------------+---------------+----------------------+-------------------+
|   CongressionalSession | PartyAffiliation   |   BillsSponsored |   BillsPassed |   PassageRate |   AvgVotesPerSenator |   PartyUnityScore |
|------------------------+--------------------+------------------+---------------+---------------+----------------------+-------------------|
|                    118 | Democratic         |                2 |             2 |           100 |                   11 |             87.5  |
|                    118 | Republican         |                2 |             2 |           100 |                   10 |             41.38 |
+------------------------+--------------------+------------------+---------------+---------------+----------------------+-------------------+


C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\2018753325.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfSenateProductivity = pd.read_sql("""


# Senator Effectiveness Profile Query

**Purpose**: Evaluates individual senators' legislative productivity, voting patterns, and party loyalty.

## Key Metrics
- `BillsSponsored`: Primary legislation introduced  
- `BillsCosponsored`: Secondary support for others' bills  
- `BillsEnacted`: Sponsored bills that became law  
- `CommitteeAssignments`: Current committee seats  
- `VotesCast`: Total participation in votes  
- `SuccessVotePercentage`: Votes aligning with majority outcomes  
- `PartyLoyaltyScore`: Votes aligned with party position  

## Technical Notes
- **Joins**: Senators → Sponsorships → Bill Status → Committees → Votes  
- `LEFT JOIN`: Includes all senators (even inactive ones)  
- `CASE WHEN`: Separates primary vs cosponsored bills  
- `NULLIF`: Safeguards division calculations  
- `ORDER BY`: Ranks by most effective legislators first  

**Use Case**: Identifies high-performing legislators and party team players.

In [26]:
def run_queries():
    conn = get_connection()
    if conn:
        try:
            # Committee Workload Query
            dfSenateIndividual = pd.read_sql("""
SELECT 
    ds.FullName,
    ds.PartyAffiliation,
    ds.StateName,
    COUNT(DISTINCT CASE WHEN fb.IsPrimarySponsor = 1 THEN fb.BillKey END) AS BillsSponsored,
    COUNT(DISTINCT CASE WHEN fb.IsPrimarySponsor = 0 THEN fb.BillKey END) AS BillsCosponsored,
    COUNT(DISTINCT CASE WHEN fbs.StatusCode = 'PASS' AND fb.IsPrimarySponsor = 1 THEN fb.BillKey END) AS BillsEnacted,
    COUNT(DISTINCT fca.CommitteeKey) AS CommitteeAssignments,
    COUNT(DISTINCT fvr.VoteRecordKey) AS VotesCast,
    CAST(COUNT(DISTINCT CASE WHEN fvr.VoteCast = 'Yea' AND fvr.VoteResult = 'Passed' THEN fvr.VoteRecordKey END) * 100.0 /
        NULLIF(COUNT(DISTINCT CASE WHEN fvr.VoteResult = 'Passed' THEN fvr.VoteRecordKey END), 0) AS DECIMAL(5,2)) AS SuccessVotePercentage,
    CAST(COUNT(DISTINCT CASE WHEN fvr.VoteCast = 'Yea' AND ds2.PartyAffiliation = ds.PartyAffiliation THEN fvr.VoteRecordKey END) * 100.0 /
        NULLIF(COUNT(DISTINCT fvr.VoteRecordKey), 0) AS DECIMAL(5,2)) AS PartyLoyaltyScore
FROM DimSenator ds
LEFT JOIN FactBillSponsorship fb ON ds.SenatorKey = fb.SenatorKey
LEFT JOIN FactBillStatus fbs ON fb.BillKey = fbs.BillKey
LEFT JOIN FactCommitteeAssignment fca ON ds.SenatorKey = fca.SenatorKey AND fca.AssignmentEndDate IS NULL
LEFT JOIN FactVoteRecord fvr ON ds.SenatorKey = fvr.SenatorKey
LEFT JOIN (
    SELECT VoteRecordKey, VoteCast, VoteResult 
    FROM FactVoteRecord
    GROUP BY VoteRecordKey, VoteCast, VoteResult
) v ON fvr.VoteRecordKey = v.VoteRecordKey
LEFT JOIN DimSenator ds2 ON fvr.SenatorKey = ds2.SenatorKey
WHERE ds.IsCurrent = 1
GROUP BY ds.FullName, ds.PartyAffiliation, ds.StateName
ORDER BY BillsEnacted DESC;
            """, conn)
            print("Senate Productivity:")
            print(tabulate(dfSenateIndividual, headers='keys', tablefmt='psql', showindex=False))
            
        finally:
            conn.close()

if __name__ == "__main__":
    run_queries()

Senate Productivity:
+-------------------+--------------------+-------------+------------------+--------------------+----------------+------------------------+-------------+-------------------------+---------------------+
| FullName          | PartyAffiliation   | StateName   |   BillsSponsored |   BillsCosponsored |   BillsEnacted |   CommitteeAssignments |   VotesCast |   SuccessVotePercentage |   PartyLoyaltyScore |
|-------------------+--------------------+-------------+------------------+--------------------+----------------+------------------------+-------------+-------------------------+---------------------|
| Alexandra Johnson | Democratic         | California  |                1 |                  3 |              1 |                      2 |          12 |                  100    |               91.67 |
| Maria Garcia      | Democratic         | New York    |                1 |                  3 |              1 |                      3 |           8 |                   85.7

C:\Users\Rajee\AppData\Local\Temp\ipykernel_52248\1430840462.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfSenateIndividual = pd.read_sql("""
